## Chapter 2-5, 1강 STT(음성→텍스트) 기초와 데이터 핸들링 — Whisper 실습

- 목표: 음성→텍스트(STT) 기본 개념과 워크플로우 이해, Whisper로 전사 및 정확도(WER) 측정
- 데이터: 짧은 음성-텍스트 pair 샘플(예: Common Voice ko 소규모 발화, 또는 제공된 예제 파일)
- 규칙(강의용): 시각화는 `matplotlib`만 사용, 불필요한 외부 의존 최소화


#### 구성
- STT 개요 및 워크플로우
- 환경 준비(`ffmpeg`, 라이브러리)와 폰트/경고 설정
- 데이터 준비: 오디오 로딩, 16kHz/모노 전처리
- Whisper 전사 데모(옵션: 언어/모델 크기)
- 정확도 평가지표(WER) 계산(참고용)



### 0. 환경 준비 및 라이브러리 임포트

- 시각화는 `matplotlib`만 사용합니다.
- STT는 `openai-whisper`, 오디오 I/O/변환은 `torchaudio`, 평가는 `jiwer`를 사용합니다.
- 한글 폰트와 경고 억제를 설정합니다.



In [ ]:
# -*- coding: utf-8 -*-
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchaudio

try:
    import whisper
    _HAS_WHISPER = True
except Exception:
    _HAS_WHISPER = False

try:
    import jiwer
    _HAS_JIWER = True
except Exception:
    _HAS_JIWER = False

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=r"Glyph.*missing from font.*", category=UserWarning)

# 한글 폰트 설정 및 마이너스 기호 깨짐 방지
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)



### 1. 데이터 준비 및 전처리
- 예제 오디오 파일 경로 지정, 로드, 16kHz 리샘플, 모노 변환
- 긴 파일은 일부 구간만 사용(강의 시간 고려, 10–15초 권장)



In [ ]:
def load_audio_mono_16k(path: str, offset_sec: float = 0.0, duration_sec: float | None = None):
    """
    오디오를 로드하여 16kHz 모노로 변환합니다.
    offset_sec부터 duration_sec 길이만 잘라 사용합니다.
    """
    if not os.path.exists(path):
        raise FileNotFoundError('오디오 파일을 찾을 수 없습니다: ' + path)

    wav, sr = torchaudio.load(path)  # (channels, time)
    # 모노 변환: 여러 채널이면 평균
    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)
    # 리샘플
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        wav = resampler(wav)
        sr = 16000
    # offset/duration 적용
    if offset_sec > 0 or duration_sec is not None:
        start = int(offset_sec * sr)
        end = start + int(duration_sec * sr) if duration_sec is not None else wav.size(1)
        wav = wav[:, start:end]
    return wav.squeeze(0), sr  # (time,), 16000

# 예시 경로 (수강생은 자신의 파일로 교체 가능)
EXAMPLE_WAV = 'example.wav'  # 존재하지 않을 경우 에러 메시지 안내



### 2. Whisper 전사 데모
- 모델 선택(`tiny`~`large-v3`/`large-v3-turbo`), 언어 지정, CPU/GPU 동작
- 길이 긴 파일은 부분 전사 또는 슬라이딩 윈도우 전략을 언급



In [ ]:
def transcribe_with_whisper(path: str, model_name: str = 'base', language: str | None = None):
    if not _HAS_WHISPER:
        raise ImportError('whisper 패키지가 설치되어 있지 않습니다. 설치: pip install -U openai-whisper')
    # Whisper는 내부적으로 ffmpeg를 사용하므로, ffmpeg 설치 필요
    model = whisper.load_model(model_name, device=DEVICE)
    # Whisper 내장 로더를 사용 (자동 리샘플/패딩 포함)
    result = model.transcribe(path, language=language)
    return result

# 데모 실행 (예시)
# try:
#     wav, sr = load_audio_mono_16k(EXAMPLE_WAV, duration_sec=15)
#     # 저장 후 Whisper에 넘기기 (메모리→파일 경로 방식)
#     tmp_path = 'tmp_16k.wav'
#     torchaudio.save(tmp_path, wav.unsqueeze(0), 16000)
#     out = transcribe_with_whisper(tmp_path, model_name='base', language='ko')
#     print('전사 결과:', out['text'][:120], '...')
# except Exception as e:
#     print('데모 실행 중 문제:', e)



### 3. 정확도 평가(WER) — 참고용
- 레퍼런스 텍스트가 있을 때만 사용
- 언어/문장부호/대소문자 정규화 후 `jiwer.wer` 계산



In [ ]:
def compute_wer(hypothesis: str, reference: str) -> float:
    if not _HAS_JIWER:
        raise ImportError('jiwer 패키지가 필요합니다. 설치: pip install jiwer')
    # 단순 정규화: 소문자/공백 정리
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
    ])
    hyp = transformation(hypothesis)
    ref = transformation(reference)
    return jiwer.wer(ref, hyp)

# 예시
# hyp = '안녕하세요 만나서 반갑습니다'
# ref = '안녕하세요 만나서 반갑습니다'
# print('WER:', compute_wer(hyp, ref))



#### 참고(부록)
- 아래에 확장 실습(Transformers/SpeechT5/Gradio)이 있었던 버전은 3강(파이프라인)으로 이동했습니다.
- 본 1강에서는 Whisper 전사와 WER까지로 최소 구성만 다룹니다.



### 4. 마무리 및 과제 제안
- 모델 크기/언어 옵션을 바꿔 전사 품질 비교
- 2~3개 파일 전사 후 간단 리포트(WER, 주관 평가)



## Chapter 2-5. STT/TTS 모델 활용 실습 — 음성-텍스트 pair 데이터

- 목표: 음성→텍스트(STT), 텍스트→음성(TTS) 기본 파이프라인을 실습하고 성능지표(WER/CER)로 평가합니다.
- 데이터: 공개 샘플(예: LibriSpeech subset) 또는 로컬 폴더(AudioFolder) 사용
- 실습 흐름: 환경 설정 → 데이터 준비 → STT 추론 → 지표 평가 → TTS 생성 → 라운드트립 데모 → Gradio 데모 → 확장/과제



### 0. 환경 설정 및 라이브러리
- CPU에서도 실행 가능하도록 기본 모델/배치로 시연합니다.
- `transformers`, `datasets[audio]`, `evaluate`, `gradio`, `torchaudio`, `soundfile` 등을 사용합니다.



In [ ]:
# 기본 설치 (인터넷 연결 필요). 이미 설치되어 있으면 건너뜀.
import sys, subprocess

def pip_install(pkgs):
    for p in pkgs:
        try:
            __import__(p.split('[')[0].split('==')[0].replace('-', '_'))
        except Exception:
            print(f'Installing {p} ...')
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', p, '--quiet'])

pip_install([
    'transformers',
    'datasets[audio]',
    'evaluate',
    'gradio',
    'torchaudio',
    'soundfile'
])

import os, random, numpy as np
import torch
import torchaudio
from datasets import load_dataset, Audio
from transformers import pipeline
import evaluate as hf_evaluate

# 디바이스 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

# 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

# 맥 한글 폰트 설정(강의 시 그래프 텍스트용, STT/TTS엔 직접 영향 없음)
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False



### 1. 데이터 준비
- 데모용: Hugging Face `librispeech_asr_dummy` 또는 `audiofolder`(로컬) 선택 가능
- 모델 샘플링레이트(보통 16kHz)에 맞춰 리샘플링합니다.



In [ ]:
from typing import List, Tuple

USE_DUMMY = True   # True: 데모용 소형 샘플, False: 로컬 오디오 폴더 사용
LOCAL_AUDIO_DIR = None  # 예: '/path/to/audiofolder' (file_name, text 메타 구성 권장)

if USE_DUMMY:
    ds = load_dataset('hf-internal-testing/librispeech_asr_dummy', 'clean', split='validation')
else:
    from datasets import load_dataset
    assert LOCAL_AUDIO_DIR is not None and os.path.exists(LOCAL_AUDIO_DIR), '로컬 폴더 경로를 설정하세요.'
    ds = load_dataset('audiofolder', data_dir=LOCAL_AUDIO_DIR, split='train')

print(ds[0])

# 샘플링레이트 맞추기(Whisper는 16kHz)
from datasets import Audio
TARGET_SR = 16000
try:
    ds = ds.cast_column('audio', Audio(sampling_rate=TARGET_SR))
except Exception:
    # 칼럼명이 다르면 유연하게 처리
    audio_col = 'audio' if 'audio' in ds.column_names else ds.column_names[0]
    ds = ds.cast_column(audio_col, Audio(sampling_rate=TARGET_SR))

# 텍스트 정리 유틸
import re

def normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9' ]+", ' ', s)  # 데모: 영문 기준 간단 정규화
    s = re.sub(r'\s+', ' ', s)
    return s

texts = []
audios = []  # (sr, np.ndarray) 형태로 수집

audio_key = 'audio' if 'audio' in ds.column_names else ds.column_names[0]
text_key  = 'text' if 'text' in ds.column_names else ('sentence' if 'sentence' in ds.column_names else None)

for ex in ds:
    aud = ex[audio_key]
    arr = aud['array']
    sr  = aud['sampling_rate']
    audios.append((sr, arr))
    if text_key and text_key in ex:
        texts.append(normalize_text(str(ex[text_key])))
    else:
        texts.append('')

print('n_samples:', len(audios), 'sr:', audios[0][0], 'shape0:', audios[0][1].shape)



### 2. STT(음성→텍스트) 추론
- Whisper 소모형 체크포인트로 CPU에서도 시연 가능 (`openai/whisper-small` 등)
- 모델 샘플링레이트에 맞춰 입력을 전달합니다.



In [ ]:
whisper_model = 'openai/whisper-small'  # CPU 데모: small/base 추천

asr = pipeline(
    task='automatic-speech-recognition',
    model=whisper_model,
    device_map='auto' if device=='cuda' else None
)

# 데이터셋 샘플 몇 개만 추론
def run_asr_batch(audios: List[Tuple[int, np.ndarray]], max_items: int = 4) -> List[str]:
    outs = []
    for (sr, arr) in audios[:max_items]:
        out = asr({'array': arr, 'sampling_rate': sr})
        text = out['text'] if isinstance(out, dict) and 'text' in out else out
        outs.append(normalize_text(str(text)))
    return outs

pred_texts = run_asr_batch(audios, max_items=min(4, len(audios)))
print('예측 텍스트:', pred_texts)
print('정답 텍스트:', texts[:len(pred_texts)])



### 3. 평가 지표: WER/CER
- `evaluate` 라이브러리의 `wer`, `cer` 또는 `xtreme_s`(WER/CER 동시) 사용
- 소문자/구두점 제거 등 정규화 여부에 따라 값이 달라집니다.



In [ ]:
wer_metric = hf_evaluate.load('wer')
cer_metric = hf_evaluate.load('cer')

# 예측/정답 쌍 정리 (빈 텍스트는 제외)
refs = [t for t in texts[:len(pred_texts)] if t]
preds = [p for p, t in zip(pred_texts, texts[:len(pred_texts)]) if t]

wer = wer_metric.compute(predictions=preds, references=refs) if refs else None
cer = cer_metric.compute(predictions=preds, references=refs) if refs else None
print({'wer': wer, 'cer': cer})



### 4. TTS(텍스트→음성) 생성
- 간편 시연: `microsoft/speecht5_tts` + `microsoft/speecht5_hifigan`
- 스피커 임베딩이 필요하므로, 제공된 샘플 임베딩을 사용합니다.



In [ ]:
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan
import soundfile as sf

# 모델/프로세서/보코더 로드
processor = SpeechT5Processor.from_pretrained('microsoft/speecht5_tts')
model = SpeechT5ForTextToSpeech.from_pretrained('microsoft/speecht5_tts')
vocoder = SpeechT5HifiGan.from_pretrained('microsoft/speecht5_hifigan')
model = model.to(device)

# 샘플 스피커 임베딩 (HF 제공 예시 파일 사용 권장)
# 간단화를 위해 랜덤 벡터(512차원) 사용 (음색 고정 X)
speaker_embeddings = torch.randn(1, 512, device=model.device)


def tts_generate(text: str, sr: int = 16000, out_wav: str | None = None) -> Tuple[int, np.ndarray]:
    inputs = processor(text=text, return_tensors='pt')
    input_ids = inputs.input_ids.to(model.device)

    with torch.no_grad():
        # vocoder를 인자로 전달하면 파형으로 변환된 torch.Tensor 반환
        audio = model.generate_speech(input_ids, speaker_embeddings, vocoder=vocoder)

    audio = audio.cpu().numpy().astype(np.float32)
    if out_wav:
        sf.write(out_wav, audio, sr)
    return sr, audio

# 데모 실행
sr_out, wav = tts_generate('hello, this is a text to speech demo using SpeechT5.')
print('TTS length:', wav.shape)



### 5. 라운드트립 데모
- STT→TTS: 음성을 텍스트로 변환 후, 그 텍스트를 다시 음성으로 합성
- TTS→STT: 텍스트를 합성한 음성을 다시 텍스트로 변환하여 손실 확인



In [ ]:
# STT → TTS
sr0, aud0 = audios[0]
stt_text = asr({'array': aud0, 'sampling_rate': sr0})
stt_text = stt_text['text'] if isinstance(stt_text, dict) else str(stt_text)
print('STT 텍스트:', stt_text)

sr_syn, wav_syn = tts_generate(stt_text)
print('합성 오디오 길이:', wav_syn.shape)

# TTS → STT (텍스트에서 합성한 음성을 다시 인식)
rt_text = asr({'array': wav_syn, 'sampling_rate': sr_syn})
rt_text = rt_text['text'] if isinstance(rt_text, dict) else str(rt_text)
print('라운드트립 텍스트:', rt_text)



### 6. Gradio 데모 UI (마이크 STT / 텍스트 TTS)
- 마이크 입력으로 STT 수행, 텍스트 입력으로 TTS 생성
- 로컬 실행 시 브라우저에서 인터랙티브 체험 가능



In [ ]:
import gradio as gr

def gr_stt(audio):
    if audio is None:
        return ''
    sr, data = audio
    out = asr({'array': data, 'sampling_rate': sr})
    return out['text'] if isinstance(out, dict) and 'text' in out else str(out)

def gr_tts(text):
    if text is None or not str(text).strip():
        return None
    return tts_generate(str(text))

with gr.Blocks() as demo:
    gr.Markdown('# STT / TTS 데모')
    with gr.Tab('STT (마이크 → 텍스트)'):
        in_audio = gr.Audio(sources=['microphone'], type='numpy')
        out_text = gr.Textbox(label='인식 텍스트')
        btn1 = gr.Button('Transcribe')
        btn1.click(gr_stt, inputs=in_audio, outputs=out_text)
    with gr.Tab('TTS (텍스트 → 오디오)'):
        in_text = gr.Textbox(label='합성할 텍스트', value='안녕하세요. STT TTS 데모입니다.')
        out_audio = gr.Audio(label='합성 오디오')
        btn2 = gr.Button('Synthesize')
        btn2.click(gr_tts, inputs=in_text, outputs=out_audio)

# 주피터에서 수동 실행 필요: demo.launch()
print('Gradio 준비 완료: demo.launch() 를 실행하세요.')



### 7. 추가 학습(개념) — LoRA/PEFT로 미세조정
- STT(Whisper) 도메인 적응: 특정 발화 스타일/도메인(콜센터/의료)을 위한 LoRA 미세조정
- TTS(SpeechT5) 화자 임베딩: 동일 화자 다량 데이터가 없을 경우, speaker embedding 활용 및 multi-speaker 학습 개념
- 데이터: 음성-텍스트 pair 정렬, 샘플링레이트/정규화 일치, 텍스트 정규화 규칙 고정
- 리소스: GPU 메모리 절약을 위한 8-bit/4-bit 로딩 + LoRA, gradient checkpointing



### 8. 참고 자료
- Transformers ASR 파이프라인: [Hugging Face Transformers pipelines](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines)
- Datasets 오디오 로딩/리샘플링: [Hugging Face Datasets Audio](https://huggingface.co/docs/datasets/audio_load)
- 평가 지표 WER/CER: [Hugging Face Evaluate WER](https://huggingface.co/spaces/evaluate-metric/wer), [CER](https://huggingface.co/spaces/evaluate-metric/cer)
- Gradio Audio 컴포넌트: [Gradio Audio Docs](https://www.gradio.app/docs/audio)
- SpeechT5: [microsoft/speecht5_tts](https://huggingface.co/microsoft/speecht5_tts), [microsoft/speecht5_hifigan](https://huggingface.co/microsoft/speecht5_hifigan)

